## このノートブックについて
このノートブックは、公式から公開されている Lux AI Season 1 Jupyter Notebook Quickstart の日本語への翻訳 + 補足情報や解釈を加えたノートブックです。

# Lux AI Season 1 Jupyter Notebook クイックスタート

Lux AI Season 1 へようこそ!

このノートブックは、Jupyter Notebooksと`kaggle-environments`パッケージを使ってボットを開発するための基本的な設定です。

Jupyter Notebooksを使わない人 や 他のプログラミング言語を使用する予定の人は、こちらの[Github](https://github.com/Lux-AI-Challenge/Lux-Design-2021)をご覧ください。

対応言語

- Python
- JavaScript
- C++
- Java
- Typescript
- Kotlin

以下は重要なリンクです。

コンペのページ: https://www.kaggle.com/c/lux-ai-2021/

Online Visualizer: https://2021vis.lux-ai.org/

Lux AIコンペの仕様: 今回のコンペのゲームルール(勝利条件などを含め)が細かく書かれているので、こちらを読むとゲーム内容が把握しやすいと思います！

https://www.lux-ai.org/specs-2021

Github: https://github.com/Lux-AI-Challenge/Lux-Design-2021

Bot API: https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits

もしDiscodeサーバーに参加していない人がいれば、参加することを強くおすすめします。 (リンク: https://discord.gg/aWJt3UAcgn )

また、最低でも Kaggleの フォーラム (https://www.kaggle.com/c/lux-ai-2021/discussion) はフォーローしておいてください。

ルールの変更、イベント、スポンサーからの紹介など、重要なお知らせを投稿します！

In [ ]:
!pip install kaggle-environments -U

In [ ]:
from kaggle_environments import make
import json

In [ ]:
# 環境を作成。 以下のようにseed と loglevelの設定を行う。もし、指定しない場合はランダムなシードが選ばれる。
# loglevel の設定
#   0 : デフォルト
#   1 : エラー用
#   2 : units の 衝突や無効なコマンドなどの 対戦への警告用 (推奨)  
#   3 : 情報(info) レベル , 4 : すべてを表示 (非推奨) 

# annotations を True にすると、 ビジュアライザーにアノテーションコマンドが描画される
# debug を True にすると、 print文が表示される
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)
# 2つのエージェント間での対戦を実行する。
# simple_agentが デフォルトのエージェント
steps = env.run(["simple_agent", "simple_agent"])

In [ ]:
# リプレイを インライン表示することも可能。見やすくするために 幅と高さをできるだけ大きく設定することを勧めている
# インタラクティブなJupyterノートブック/Kaggleノートブックモード以外で表示すると、一部が切れた表示になる場合がある
# また、このレンダリングセルの出力を閉じておくと、ノートブックの動作が重くなるかもしれないので注意
env.render(mode="ipython", width=1200, height=800)

In [ ]:
# Kaggle の Notebookを使っている場合は、このコードを実行
!cp -r ../input/lux-ai-2021/* .
# ローカルで作業する場合は、ここから `simple/lux` フォルダをダウンロード  https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits/python
# また、Pythonボットを使ったローカル開発では、ここに書かれている指示に従うことをお勧めする

In [ ]:
%%writefile agent.py
# メインエージェント のコードをファイルに書き込んで、後でsubmissionできるようにするために上記のコードを追加

# kaggle-environments のための import
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math

# global変数として game_stateを宣言することで、ターンをまたいでも状態を持続できるようにし、常に初期化をし直す必要がないようにする
game_state = None

# 基本のエージェントの定義
# 現時点では、このエージェントは何もしない (実際には、闇に飲み込まれる前の少しの間は生き延びることができるが)
def agent(observation, configuration):
    global game_state

    ### 編集しない部分 ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AIのコードはこれ以下で書く! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    return actions

In [ ]:
# 空のエージェントを使って別の対戦を実行する
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2}, debug=True)
steps = env.run(["./agent.py", "./agent.py"])

In [ ]:
# fileとして リプレイが保存される。
# もし、ローカルで作業をしている場合はカレントディレクトリに保存される。 
# Kaggleでは、画面右側のデータパネルの output セクションを開いて、ダウンロードすることができる
replay = env.toJSON()
with open("replay.json", "w") as f:
    json.dump(replay, f)
# このリプレイは、ここで見ることができる https://2021vis.lux-ai.org/

## Submission を作成
main.py（とagent.py）を一番上の階層においたtar.gzファイルを作成する必要がある。

そして、作成できたらそれをアップロードをする。

In [ ]:
!tar -czf submission.tar.gz *

## Submit

submit の 手順

1. /kaggle/working フォルダを開き、submission.tar.gz を見つけて、そのファイルをダウンロード

2. https://www.kaggle.com/c/lux-ai-2021/ の「MySubmissions」タブに移動して、投稿をアップロードする

### 注意点

- このファイルは自分自身に対して検証用の対戦を行い、成功すると他のプレイヤーの提出物と自動的にマッチングされる

- 新しい投稿は、古い投稿よりも優先的にゲームに採用される

- 1日にsubmitできる数は5つと制限があるので、submissionする前にローカルでボットをテストすることを強くお勧めする。



## 追加でチェックすべきこと

### 1. Bot APIについて

Bot API について確認を行う → https://github.com/Lux-AI-Challenge/Lux-Design-2021/tree/master/kits

このドキュメントでは、スターターキットのファイルを使ってできることや、リプレイに直接注釈を加えることができる annotation debug commands の使い方が説明されている (線を引いたり、円を描いたり など)

This documents what you can do using the starter kit files in addition to telling you how to use the annotation debug commands that let you annotate directly on a replay (draw lines, circle things etc.)

### 2. online replay viewer
先ほど作成したJSONファイルのようなリプレイファイルを、online replay viewer で見ることができる→ https://2021vis.lux-ai.org/

### 3.ローカルバージョン (replay viewer)
replay viewer の ローカル(高速)バージョンのインストールはこちらから → https://github.com/Lux-AI-Challenge/Lux-Viewer-2021

## CLI Tool

対戦を実行するために使用できる別の CLIツールも用意されている。

Jupyter Notebookを使っている人は、このクイックスタートノートブックを使うことを勧める。

それ以外の人 (pythonを使う人も含め) は、https://github.com/Lux-AI-Challenge/Lux-Design-2021 に従うこと

CLIツールを使用する他の利点としては、より細かな "stateless"のリプレイを生成し、様々なランキングアルゴリズムでランク付けされた複数のボットのミニリーダーボードを実行することができるところにあります！

## おわりに
Lux AI Season 1 Jupyter Notebook Quickstart について日本語版を作成していきました。もし訳や解釈等について、ご指摘・ご意見等あれば教えてください
